In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import pymongo
from splinter import Browser
from webdriver_manager.chrome import ChromeDriverManager

In [2]:
# Splinter
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)



====== WebDriver manager ======
Current google-chrome version is 91.0.4472
Get LATEST driver version for 91.0.4472
Driver [C:\Users\Robmir\.wdm\drivers\chromedriver\win32\91.0.4472.101\chromedriver.exe] found in cache


In [3]:
#Scrape News
url = 'https://redplanetscience.com/'
browser.visit(url)
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

In [4]:
news_title = soup.find('div', class_='content_title').text
print(news_title)

Mars 2020 Unwrapped and Ready for More Testing


In [5]:
news_teaser = soup.find('div', class_='article_teaser_body').text
print(news_teaser)

In time-lapse video, bunny-suited engineers remove the inner layer of protective foil on NASA's Mars 2020 rover after it was relocated for testing.


In [6]:
#Scrape images
url = 'https://spaceimages-mars.com/'
browser.visit(url)
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

In [7]:
relative_image_path = soup.find_all('img')[1]["src"]
featured_image_url = url + relative_image_path
featured_image_url

'https://spaceimages-mars.com/image/featured/mars3.jpg'

In [8]:
#Scrape Facts
url = 'https://galaxyfacts-mars.com/'
tables = pd.read_html(url)
tables

[                         0                1                2
 0  Mars - Earth Comparison             Mars            Earth
 1                Diameter:         6,779 km        12,742 km
 2                    Mass:  6.39 × 10^23 kg  5.97 × 10^24 kg
 3                   Moons:                2                1
 4       Distance from Sun:   227,943,824 km   149,598,262 km
 5          Length of Year:   687 Earth days      365.24 days
 6             Temperature:     -87 to -5 °C      -88 to 58°C,
                       0                              1
 0  Equatorial Diameter:                       6,792 km
 1       Polar Diameter:                       6,752 km
 2                 Mass:  6.39 × 10^23 kg (0.11 Earths)
 3                Moons:          2 ( Phobos & Deimos )
 4       Orbit Distance:       227,943,824 km (1.38 AU)
 5         Orbit Period:           687 days (1.9 years)
 6  Surface Temperature:                   -87 to -5 °C
 7         First Record:              2nd millennium BC

In [11]:
df = tables[0]
df.columns = df.iloc[0]
comparison = df[1:]
comparison.set_index('Mars - Earth Comparison')

,Mars,Earth
Mars - Earth Comparison,,
Diameter:,"6,779 km","12,742 km"
Mass:,6.39 × 10^23 kg,5.97 × 10^24 kg
Moons:,2,1
Distance from Sun:,"227,943,824 km","149,598,262 km"
Length of Year:,687 Earth days,365.24 days
Temperature:,-87 to -5 °C,-88 to 58°C


In [13]:
html_table = comparison.to_html()
html_table.replace('\n', '')
comparison.to_html('table.html')

In [21]:
#Scrape Hemispheres
url = 'https://marshemispheres.com/'
browser.visit(url)
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

In [22]:
h_names = soup.find_all('div', class_='item')

In [23]:
hemispheres = []

for hemis in h_names: 
    itemtitle = hemis.find('div', class_='description').find('a').find('h3').text
    baseurl = hemis.find('a')['href']
    usgs_url = 'https://marshemispheres.com/'
    image_url = usgs_url + baseurl
    browser.visit(image_url)
    html = browser.html
    soup = BeautifulSoup(html, 'html.parser')
    image = soup.find('div', class_='downloads').find('ul').find('li').find('a')['href']
    final_image = usgs_url + image
    hemispheres.append({'title':itemtitle, 'img_url':final_image})

In [24]:
print(hemispheres)

[{'title': 'Cerberus Hemisphere Enhanced', 'img_url': 'https://marshemispheres.com/images/full.jpg'}, {'title': 'Schiaparelli Hemisphere Enhanced', 'img_url': 'https://marshemispheres.com/images/schiaparelli_enhanced-full.jpg'}, {'title': 'Syrtis Major Hemisphere Enhanced', 'img_url': 'https://marshemispheres.com/images/syrtis_major_enhanced-full.jpg'}, {'title': 'Valles Marineris Hemisphere Enhanced', 'img_url': 'https://marshemispheres.com/images/valles_marineris_enhanced-full.jpg'}]


In [27]:
mars_dict = {'news_title': news_title,
        'news_teaser': news_teaser,
        'featured_image_url': featured_image_url,
        'mars_facts': html_table,
        'hemisphere_images': hemispheres}

In [28]:
mars_dict

{'featured_image_url': 'https://spaceimages-mars.com/image/featured/mars3.jpg',
 'hemisphere_images': [{'img_url': 'https://marshemispheres.com/images/full.jpg',
   'title': 'Cerberus Hemisphere Enhanced'},
  {'img_url': 'https://marshemispheres.com/images/schiaparelli_enhanced-full.jpg',
   'title': 'Schiaparelli Hemisphere Enhanced'},
  {'img_url': 'https://marshemispheres.com/images/syrtis_major_enhanced-full.jpg',
   'title': 'Syrtis Major Hemisphere Enhanced'},
  {'img_url': 'https://marshemispheres.com/images/valles_marineris_enhanced-full.jpg',
   'title': 'Valles Marineris Hemisphere Enhanced'}],
 'mars_facts': '<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>Mars - Earth Comparison</th>\n      <th>Mars</th>\n      <th>Earth</th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>1</th>\n      <td>Diameter:</td>\n      <td>6,779 km</td>\n      <td>12,742 km</td>\n    </tr>\n    <tr>\n      <th>2</th>\n      <t

In [29]:
browser.quit()